# Testing Dataset Creation

Create testing pth files from the data.mat files of processed scenes, one pth file per scene.

In [ ]:
import h5py
import os
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy.interpolate import interp1d
import sys
import copy 
import torch

sys.path.append('../functions/')
from helpers import crop_center_array

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
scene_names_test = [
        '0625.0928.CheckerEKE', 
        '0625.0938.CheckerLED', 
        '0625.0955.HHPainting', 
        '0625.1049.MatroshkaFamily', 
        '0625.1331.FeathersZoom',
        '0628.1131.Flowers',
        '0628.1229.Feathers',
        '0628.1303.Painting',
        '0628.1316.Chopper',
        '0628.1332.SpectralonEKE',
        '0628.1332.SpectralonLED',
        '0723.1436.ButterflyWhite', 
        '0723.1419.ButterflyOrange',
        '0729.1303.ButterflyBrown',
        '0729.1320.ButterflyOrangev2',
        '0729.1337.ButterflyBlue',
        '0729.1401.ButterflyTransp',
        '0730.1448.TheBulb_16',
        '0730.1448.TheBulb_33',
        '0730.1529.Plants',
        '0730.1551.CheckerCFL',
        '0803.1155.Plants',
        '0803.1155.Plants2',
        '0806.1542.Plants3'
]

PROCESSEDDATA_DIR = 'Box/data/Processed/'
SAVEPTHFILE_DIR = '../data/restore/'

# Choose scene index one by one manually.
scene_name = scene_names_test[0]
save_pthfile_test = scene_name + '_data4_1024x1024.pth'

crop_size = 1024

data_matfile = PROCESSEDDATA_DIR + '/' + scene_name + '/data.mat'
data = sio.loadmat(data_matfile)

assort_sim = data['assort_sim'].astype(float)
assort_meas = data['assort_meas'].astype(float)
assort_index = data['assort_index'].astype(float)
guide_image = data['guide'].astype(float)

if crop_size:
    assort_sim = crop_center_array(assort_sim, crop_size, crop_size)
    assort_meas = crop_center_array(assort_meas, crop_size, crop_size)
    assort_index = crop_center_array(assort_index, crop_size, crop_size)
    guide_image = crop_center_array(guide_image, crop_size, crop_size)

if len(assort_sim.shape) == 2: assort_sim = np.expand_dims(assort_sim, 2)
if len(assort_meas.shape) == 2: assort_meas = np.expand_dims(assort_meas, 2)
if len(assort_index.shape) == 2: assort_index = np.expand_dims(assort_index, 2)

assort_sim = assort_sim.transpose(2,0,1) 
assort_meas = assort_meas.transpose(2,0,1) 
assort_index = assort_index.transpose(2,0,1) 
guide_image = guide_image.transpose(2,0,1) 

assort_sim_t = torch.tensor(assort_sim, dtype=torch.float)
assort_meas_t = torch.tensor(assort_meas, dtype=torch.float)
assort_index_t = torch.tensor(assort_index, dtype=torch.float)
guide_image_t = torch.tensor(guide_image, dtype=torch.float)
    
torch.save({'assort_sim': assort_sim_t.detach().clone(),
            'assort_meas': assort_meas_t.detach().clone(),
            'assort_index': assort_index_t.detach().clone(),
            'guide_image': guide_image_t.detach().clone()
           }, 
           os.path.join(SAVEPTHFILE_DIR, save_pthfile_test))
